In [7]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz


In [8]:
df = pd.read_csv("stores.csv")

print(df.head())


   StoreKey    Country                         State  Square Meters  Open Date
0         1  Australia  Australian Capital Territory          595.0   1/1/2008
1         2  Australia            Northern Territory          665.0  1/12/2008
2         3  Australia               South Australia         2000.0   1/7/2012
3         4  Australia                      Tasmania         2000.0   1/1/2010
4         5  Australia                      Victoria         2000.0  12/9/2015


In [9]:
encoding_lookup = {
    "Franche-ComtÃ©": "Franche-Comté",
    "La RÃ©union": "La Réunion",
    "Freistaat ThÃ¼ringen": "Freistaat Thüringen"
}

df["State"] = df["State"].replace(encoding_lookup)


In [10]:
state_master = pd.DataFrame({
    "Country": [
        "France","France","France","France",
        "Germany","Germany","Germany","Germany",
        "Australia","Australia","Australia","Australia",
        "United Kingdom","United Kingdom",
        "United States","United States"
    ],
    "State_Clean": [
        "Franche-Comté","La Réunion","Basse-Normandie","Corse",
        "Freistaat Thüringen","Brandenburg","Berlin","Hamburg",
        "Australian Capital Territory","Western Australia",
        "South Australia","Victoria",
        "Belfast","Ayrshire",
        "Alaska","Washington DC"
    ]
})


In [11]:
def fuzzy_match_state(country, state, master_df, threshold=85):
    if pd.isna(state):
        return state, 0

    choices = master_df[
        master_df["Country"] == country
    ]["State_Clean"].tolist()

    if not choices:
        return state, 0

    match, score, _ = process.extractOne(
        state,
        choices,
        scorer=fuzz.token_sort_ratio
    )

    if score >= threshold:
        return match, score
    else:
        return state, score


In [12]:
df[["State_Standardized", "MatchScore"]] = df.apply(
    lambda row: fuzzy_match_state(
        row["Country"],
        row["State"],
        state_master
    ),
    axis=1,
    result_type="expand"
)


In [13]:
df["NeedsManualReview"] = df["MatchScore"] < 85


In [15]:
df["Open Date"] = pd.to_datetime(
    df["Open Date"],
    errors="coerce",
    #infer_datetime_format=True
)

df["Open Date"] = df["Open Date"].dt.strftime("%Y-%m-%d")


In [16]:
df["StoreType"] = np.where(
    df["Country"] == "Online",
    "Online",
    "Physical"
)


In [17]:
df["Square Meters"] = pd.to_numeric(
    df["Square Meters"],
    errors="coerce"
)

# Missing size
df["MissingSizeFlag"] = df["Square Meters"].isna()

# Suspicious default size
df["IsDefaultSize"] = df["Square Meters"] == 2000


In [18]:
df["IsSpecialStore"] = df["StoreKey"] == 0


In [20]:
final_cols = [
    "StoreKey",
    "Country",
    "State_Standardized",
    "Square Meters",
    "Open Date",
    "StoreType",
    "MatchScore",
    "NeedsManualReview",
    "MissingSizeFlag",
    "IsDefaultSize",
    "IsSpecialStore"
]

df_clean = df[final_cols]

df_clean.to_csv("stores_cleaned.csv", index=False)


In [ ]:
#sales data cleaning

In [25]:
# Load CSV
df = pd.read_csv("Sales.csv")

# Quick overview
print(df.shape)
print(df.head())

# Standardize column names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

print(df.columns)

# Convert date columns
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")
df["delivery_date"] = pd.to_datetime(df["delivery_date"], errors="coerce")

#detect missing values
missing_report = df.isnull().sum().reset_index()
missing_report.columns = ["column", "missing_count"]

print(missing_report)


#detect invalid store keys
invalid_storekey_df = df[df["storekey"] == 0]

print("Invalid StoreKey rows:", len(invalid_storekey_df))

#bussiness rule validation
invalid_dates_df = df[
    (df["delivery_date"].notna()) &
    (df["delivery_date"] < df["order_date"])
]

print("Delivery before Order:", len(invalid_dates_df))

#quantity <=0(invalid sales)
invalid_quantity_df = df[df["quantity"] <= 0]

print("Invalid Quantity rows:", len(invalid_quantity_df))


#duplicate records
duplicate_rows = df.duplicated().sum()

print("Duplicate rows:", duplicate_rows)

#currency code validation
valid_currencies = {"USD", "EUR", "GBP", "CAD", "AUD"}

invalid_currency_df = df[~df["currency_code"].isin(valid_currencies)]

print("Invalid currency rows:", len(invalid_currency_df))


#data_summary report
data_quality_summary = {
    "total_rows": len(df),
    "missing_delivery_date": df["delivery_date"].isna().sum(),
    "invalid_storekey": (df["storekey"] == 0).sum(),
    "delivery_before_order": len(invalid_dates_df),
    "invalid_quantity": len(invalid_quantity_df),
    "duplicate_rows": duplicate_rows,
    "invalid_currency": len(invalid_currency_df)
}

dq_report = pd.DataFrame(
    data_quality_summary.items(),
    columns=["issue", "count"]
)

print(dq_report)

#tagging anomallies in data

df["anomaly_flag"] = "VALID"

df.loc[df["delivery_date"].isna(), "anomaly_flag"] = "MISSING_DELIVERY_DATE"
df.loc[df["storekey"] == 0, "anomaly_flag"] = "INVALID_STOREKEY"
df.loc[df["quantity"] <= 0, "anomaly_flag"] = "INVALID_QUANTITY"
df.loc[
    (df["delivery_date"].notna()) &
    (df["delivery_date"] < df["order_date"]),
    "anomaly_flag"
] = "DELIVERY_BEFORE_ORDER"

#save structured data
# Valid data
clean_df = df[df["anomaly_flag"] == "VALID"]

# Anomalies
anomaly_df = df[df["anomaly_flag"] != "VALID"]

clean_df.to_csv("clean_sales.csv", index=False)
anomaly_df.to_csv("sales_anomalies.csv", index=False)



(55071, 9)
   Order Number  Line Item  Order Date Delivery Date  CustomerKey  StoreKey  \
0        366000          1  01-01-2016           NaN       265598        10   
1        366001          1  01-01-2016     1/13/2016      1269051         0   
2        366001          2  01-01-2016     1/13/2016      1269051         0   
3        366002          1  01-01-2016    01-12-2016       266019         0   
4        366002          2  01-01-2016    01-12-2016       266019         0   

   ProductKey  Quantity Currency Code  
0        1304         1           CAD  
1        1048         2           USD  
2        2007         1           USD  
3        1106         7           CAD  
4         373         1           CAD  
Index(['order_number', 'line_item', 'order_date', 'delivery_date',
       'customerkey', 'storekey', 'productkey', 'quantity', 'currency_code'],
      dtype='object')
          column  missing_count
0   order_number              0
1      line_item              0
2     order